In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3

In [2]:
nppes_cols = [ 'NPI', 
              'Entity Type Code', 
              'Provider Organization Name (Legal Business Name)',
              'Provider Name Prefix Text',
              'Provider First Name',
              'Provider Middle Name',
              'Provider Last Name (Legal Name)', 
              'Provider Name Suffix Text',
              'Provider Credential Text',
              'Provider First Line Business Practice Location Address',
              'Provider Second Line Business Practice Location Address',
              'Provider Business Practice Location Address City Name',
              'Provider Business Practice Location Address State Name',
              'Provider Business Practice Location Address Postal Code',
              'Healthcare Provider Taxonomy Code_1',
              'Healthcare Provider Primary Taxonomy Switch_1',
              'Healthcare Provider Taxonomy Code_2',
              'Healthcare Provider Primary Taxonomy Switch_2',
              'Healthcare Provider Taxonomy Code_3',
              'Healthcare Provider Primary Taxonomy Switch_3',
              'Healthcare Provider Taxonomy Code_4',
              'Healthcare Provider Primary Taxonomy Switch_4',
              'Healthcare Provider Taxonomy Code_5',
              'Healthcare Provider Primary Taxonomy Switch_5',
              'Healthcare Provider Taxonomy Code_6',
              'Healthcare Provider Primary Taxonomy Switch_6',
              'Healthcare Provider Taxonomy Code_7',
              'Healthcare Provider Primary Taxonomy Switch_7',
              'Healthcare Provider Taxonomy Code_8',
              'Healthcare Provider Primary Taxonomy Switch_8',
              'Healthcare Provider Taxonomy Code_9',
              'Healthcare Provider Primary Taxonomy Switch_9',
              'Healthcare Provider Taxonomy Code_10',
              'Healthcare Provider Primary Taxonomy Switch_10',
              'Healthcare Provider Taxonomy Code_11',
              'Healthcare Provider Primary Taxonomy Switch_11',
              'Healthcare Provider Taxonomy Code_12',
              'Healthcare Provider Primary Taxonomy Switch_12',
              'Healthcare Provider Taxonomy Code_13',
              'Healthcare Provider Primary Taxonomy Switch_13',
              'Healthcare Provider Taxonomy Code_14',
              'Healthcare Provider Primary Taxonomy Switch_14',
              'Healthcare Provider Taxonomy Code_15',
              'Healthcare Provider Primary Taxonomy Switch_15']


In [4]:
nppes=pd.read_csv("../hop_team-the_cheesy_nachos/data/npidata_pfile_20050523-20220213.csv", 
              usecols = nppes_cols,
              dtype = str,
              chunksize = 100000,
              low_memory = False)

In [6]:
tn_providers = pd.DataFrame()

for chunk in tqdm(nppes):
    filtered_chunk = chunk[nppes_cols].query('`Provider Business Practice Location Address State Name` == "TN"')
    tn_providers =tn_providers.append(filtered_chunk)

tn_providers.columns = ['npi', 
                        'entity_type_code', 
                        'organization_name',
                        'name_prefix', 
                        'first_name', 
                        'middle_name',
                        'last_name',
                        'name_suffix',
                        'credentials', 
                        'address_first_line', 
                        'address_second_line', 
                        'city',
                        'state', 
                        'zipcode',
                        'taxonomy_code_1',
                        'primary_taxonomy_switch_1',
                        'taxonomy_code_2',
                        'primary_taxonomy_switch_2',
                         'taxonomy_code_3',
                        'primary_taxonomy_switch_3',
                         'taxonomy_code_4',
                        'primary_taxonomy_switch_4',
                         'taxonomy_code_5',
                        'primary_taxonomy_switch_5',
                         'taxonomy_code_6',
                        'primary_taxonomy_switch_6',
                         'taxonomy_code_7',
                        'primary_taxonomy_switch_7',
                         'taxonomy_code_8',
                        'primary_taxonomy_switch_8',
                         'taxonomy_code_9',
                        'primary_taxonomy_switch_9',
                         'taxonomy_code_10',
                        'primary_taxonomy_switch_10',
                         'taxonomy_code_11',
                        'primary_taxonomy_switch_11',
                         'taxonomy_code_12',
                        'primary_taxonomy_switch_12',
                         'taxonomy_code_13',
                        'primary_taxonomy_switch_13',
                         'taxonomy_code_14',
                        'primary_taxonomy_switch_14',
                         'taxonomy_code_15',
                        'primary_taxonomy_switch_15']



0it [00:00, ?it/s]

In [11]:
tn_providers['zipcode'] = tn_providers['zipcode'].astype(str).str[:5]


cbsa = pd.read_excel('../hop_team-the_cheesy_nachos/data/ZIP_CBSA_122021.xlsx', dtype = {'zip': str, 'cbsa': str}) \
    .query('usps_zip_pref_state == "TN"') \
    .sort_values('tot_ratio', ascending = False) \
    .drop_duplicates(subset = 'zip') \
    [['zip', 'cbsa', 'usps_zip_pref_city', 'usps_zip_pref_state']]


cbsa.columns = ['zipcode', 'cbsa', 'cbsa_city', 'cbsa_state']


tn_providers = tn_providers.merge(cbsa, on = 'zipcode')

In [12]:
tn_providers.head()

,npi,entity_type_code,organization_name,name_prefix,first_name,middle_name,last_name,name_suffix,credentials,address_first_line,...,primary_taxonomy_switch_12,taxonomy_code_13,primary_taxonomy_switch_13,taxonomy_code_14,primary_taxonomy_switch_14,taxonomy_code_15,primary_taxonomy_switch_15,cbsa,cbsa_city,cbsa_state
0,1679776678,2,"OFFICE OF INTERNAL MEDICINE, PC",NaN,NaN,NaN,NaN,NaN,NaN,1325 EASTMORELAND AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32820,MEMPHIS,TN
1,1881897635,1,NaN,NaN,SHERYL,A,EBERT,NaN,"PHD, MSPH",1030 JEFFERSON AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32820,MEMPHIS,TN
2,1265636443,1,NaN,DR.,LAURA,LYNNE,ENGBRETSON,NaN,M.D.,1211 UNION AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32820,MEMPHIS,TN
3,1548464886,2,"JAMES B. COCHRAN, DDS, PC",NaN,NaN,NaN,NaN,NaN,NaN,1711 MADISON AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32820,MEMPHIS,TN
4,1407050230,1,NaN,DR.,THOMAS,MEADE,HIGLEY,NaN,M.D.,1265 UNION AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32820,MEMPHIS,TN


In [14]:
id_cols = ['npi', 
           'entity_type_code', 
           'organization_name',
           'name_prefix', 
           'first_name', 
           'middle_name',
           'last_name',
           'name_suffix',
           'credentials', 
           'address_first_line', 
           'address_second_line', 
           'city',
           'state', 
           'zipcode',
           'cbsa', 
           'cbsa_city', 
           'cbsa_state']


var_cols = ['taxonomy_code', 
            'primary_taxonomy_switch']

tn_providers=pd.wide_to_long(tn_providers, stubnames=var_cols, i = id_cols, j= '', sep = '_') \
             .sort_values('npi') \
             .query('primary_taxonomy_switch == "Y"') \
             .reset_index() \
             .drop(['' , 'primary_taxonomy_switch'], axis = 1)


In [15]:
tn_providers.head()


,npi,entity_type_code,organization_name,name_prefix,first_name,middle_name,last_name,name_suffix,credentials,address_first_line,address_second_line,city,state,zipcode,cbsa,cbsa_city,cbsa_state,taxonomy_code
0,1003000282,1,NaN,NaN,ROSIE,K,BLAKEMORE,NaN,FNP,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,TN,37243,34980,NASHVILLE,TN,363LF0000X
1,1003001066,2,WOMENS HEALTH SERVICES CHATTANOOGA PC,NaN,NaN,NaN,NaN,NaN,NaN,6845 MOUNTAIN VIEW RD,NaN,OOLTEWAH,TN,37363,16860,OOLTEWAH,TN,174400000X
2,1003002254,2,WALGREEN CO.,NaN,NaN,NaN,NaN,NaN,NaN,5104 BOBBY HICKS HWY,NaN,GRAY,TN,37615,27740,JOHNSON CITY,TN,333600000X
3,1003002734,1,NaN,NaN,JONATHAN,NaN,MCCOLLUM,NaN,LCSW,210 SIMMONS ST,NaN,MARYVILLE,TN,37801,28940,MARYVILLE,TN,1041C0700X
4,1003003245,1,NaN,MRS.,REBEKAH,GOOD,DOREAU,NaN,M.A.,6110 SHALLOWFORD RD,SUITE B,CHATTANOOGA,TN,37421,16860,CHATTANOOGA,TN,101YM0800X


In [19]:
nucc = pd.read_csv('../hop_team-the_cheesy_nachos/data/nucc_taxonomy_220.csv').drop(['Definition', 'Notes'], axis = 1)

nucc.columns = ['taxonomy_code',
                'taxonomy_grouping',
                'taxonomy_classification',
                'taxonomy_specialization',
                'taxonomy_display_name',
                'taxonomy_section']

In [20]:
nucc.head()

,taxonomy_code,taxonomy_grouping,taxonomy_classification,taxonomy_specialization,taxonomy_display_name,taxonomy_section
0,193200000X,Group,Multi-Specialty,NaN,Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Clinical & Laboratory Immunology (Allergy & Im...,Individual


In [21]:
tn_providers = tn_providers.merge(nucc, on = 'taxonomy_code')

In [22]:
tn_providers.head()

,npi,entity_type_code,organization_name,name_prefix,first_name,middle_name,last_name,name_suffix,credentials,address_first_line,...,zipcode,cbsa,cbsa_city,cbsa_state,taxonomy_code,taxonomy_grouping,taxonomy_classification,taxonomy_specialization,taxonomy_display_name,taxonomy_section
0,1003000282,1,NaN,NaN,ROSIE,K,BLAKEMORE,NaN,FNP,TENNESSEE PRISON FOR WOMEN,...,37243,34980,NASHVILLE,TN,363LF0000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family,Family Nurse Practitioner,Individual
1,1003006396,1,NaN,NaN,MICHELLE,L,RUSH,NaN,"MSN, APRN, BC",333 S BELLEVUE BLVD,...,38104,32820,MEMPHIS,TN,363LF0000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family,Family Nurse Practitioner,Individual
2,1003048810,1,NaN,MRS.,PATRICIA,M,HUDDLESTON,NaN,NP,1940 ALCOA HWY,...,37920,28940,KNOXVILLE,TN,363LF0000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family,Family Nurse Practitioner,Individual
3,1003072273,1,NaN,MS.,SANDRA,CANDECE,HUGGINS,NaN,FNP-BC,1874 DECATUR PIKE,...,37303,11940,ATHENS,TN,363LF0000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family,Family Nurse Practitioner,Individual
4,1003089129,1,NaN,NaN,TABITHA,STARR,DAVIDSON,NaN,NP,105 W STONE DR,...,37660,28700,KINGSPORT,TN,363LF0000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,Family,Family Nurse Practitioner,Individual


In [23]:
tn_providers.to_csv('../hop_team-the_cheesy_nachos/data/tn_providers.csv', index = False)

In [25]:
hop_teaming=pd.read_csv("../hop_team-the_cheesy_nachos/data/DocGraph_Hop_Teaming_2018.csv",
           dtype = {'from_npi' : str, 'to_npi': str},
           chunksize = 100000,
           low_memory = False)         

entity_type = dict(tn_providers[['npi', 'entity_type_code']].values)

tn_hop_teaming = pd.DataFrame()

for chunk in tqdm(hop_teaming):
    filtered_chunk = chunk[(chunk['from_npi'].map(entity_type) == '1') &
                           (chunk['to_npi'].map(entity_type) == '2') &
                           (chunk['transaction_count'] >= 50) &
                           (chunk['average_day_wait'] < 50)]
    tn_hop_teaming = tn_hop_teaming.append(filtered_chunk)
           

0it [00:00, ?it/s]

In [26]:
tn_hop_teaming.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
7571,1508264144,1730371865,56,158,6.171,12.735
28864,1508003815,1740527753,68,312,0.000,0.000
68044,1508027277,1760888549,151,245,0.000,0.000
83524,1497990170,1770807919,50,51,19.314,29.137
107130,1508168303,1780986257,85,107,22.290,49.446


In [ ]:
#tn_hop_teaming.to_csv('../hop_team-the_cheesy_nachos/data/tn_hop_teaming.csv', index = False)